In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import featuretools as ft
from fbprophet import Prophet

In [7]:
houses = pd.read_csv("./../src/data/raw/assessor.csv")
houses.head()

,Closed Roll Year,Property Location,Parcel Number,Block,Lot,Volume Number,Use Code,Use Definition,Property Class Code,Property Class Code Definition,...,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,Supervisor District,Analysis Neighborhood,the_geom,Row ID
0,2007.0,0000 0000 0000,1378007,1378,007,10,MISC,Miscellaneous/Mixed-Use,V,Vacant Lot,...,0,0,0,1.0,1D,Lake --The Presidio,1,Outer Richmond,POINT (-122.478111692066 37.785130177493),20071378007
1,2008.0,0000 0000 0000,1345001,1345,001,10,MISC,Miscellaneous/Mixed-Use,V,Vacant Lot,...,0,0,0,1.0,1D,Lake --The Presidio,2,Inner Richmond,POINT (-122.46850833548 37.787296873264),20081345001
2,2008.0,0000 0000 0000,5607046,5607,046,33,MISC,Miscellaneous/Mixed-Use,V,Vacant Lot,...,0,0,0,9.0,9A,Bernal Heights,9,Portola,POINT (-122.408022500468 37.737569794488),20085607046
3,2009.0,0000 0297 BALTIMORE WY0000,6499040,6499,040,38,SRES,Single Family Residential,D,Dwelling,...,45734,22044,0,10.0,10B,Croker Amazon,11,Excelsior,POINT (-122.437016077245 37.708207457788),20096499040
4,2010.0,0000 1715 SITUS TO BE ASSIGNEDST0000,0663054,0663,054,5,COMM,Commercial Misc,E,Schools,...,0,0,0,6.0,6C,Lower Pacific Heights,5,Pacific Heights,POINT (-122.427448868223 37.78845382468),20100663054


In [90]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
my_data = houses.loc[houses['Assessor Neighborhood'] == "Lake --The Presidio" ]
final = my_data[['Closed Roll Year', 'Assessed Improvement Value', 'Assessed Land Value']]
final.rename(columns={ "Closed Roll Year": "my_index"}, inplace = True)

# final = np.around(final[['my_index']],decimals= 0)


final = final.groupby('my_index').mean()
final['y'] = final['Assessed Improvement Value'] + final['Assessed Land Value']
final = np.around(final[['y']],decimals= 3)
final['ds'] = final.index
# final = final.astype(np.float64)
final['ds'] = le.fit_transform(final['ds'].astype(int))
# final['ds']= pd.to_datetime(final['ds'], format='%Y')

# final['ds'].astype(str).apply(lambda x: pd.to_datetime(x, format='%Y'))

final

,y,ds
my_index,,
"2,007","643,713",0
"2,008","674,920",1
"2,009","720,429",2
"2,010","730,010",3
"2,011","746,153",4
"2,012","771,094",5
"2,013","801,097",6
"2,014","838,898",7
"2,016","935,868",8


In [85]:
# final.astype(str).apply(lambda x: pd.to_datetime(x, format='%Y'))
# final = final.groupby('my_index').mean()
# final['y'] = final['Assessed Improvement Value'] + final['Assessed Land Value']
# final = np.around(final[['y']],decimals= 3)
# final['ds'] = final.index
# final = final.astype(np.float64)

In [86]:
m = Prophet(yearly_seasonality = True,weekly_seasonality=False,daily_seasonality=False, seasonality_prior_scale=0.1)
m.fit(final)
future = m.make_future_dataframe(periods = 2, freq= 'Y')
forecast = m.predict(future)
pd.plotting.register_matplotlib_converters()

m.plot_components(forecast)
fig1 = m.plot(forecast)

ValueError: day is out of range for month